In [1]:
import pandas as pd
import os
import datetime
import pickle
import re
import time

## 1. Import articles with area codes

In [2]:
# Reg relevant articles
df_regSentsExpandRelevant=pd.read_pickle('/home/ec2-user/SageMaker/New Uncertainty/Categorical Index/RegSentsExpand_NounChunks_Area.pkl')
print(df_regSentsExpandRelevant.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493418 entries, 0 to 493417
Data columns (total 15 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   ID                           493418 non-null  int64 
 1   RegSentsExpand               493418 non-null  object
 2   NounChunksMatch              493418 non-null  int64 
 3   NounChunkMatchWords          493418 non-null  object
 4   NounChunkMatchWordsFiltered  493418 non-null  object
 5   NounChunkMatchFiltered       493418 non-null  int64 
 6   DominantNounChunk            493418 non-null  object
 7   DominantNounChunkArea        493418 non-null  object
 8   AllAreas                     493418 non-null  object
 9   AreaCount                    493418 non-null  object
 10  DominantArea                 493418 non-null  object
 11  AllDistinctAreas             493418 non-null  object
 12  UniqueDistinctAreas          493418 non-null  object
 13  DistinctAreaCo

In [3]:
# All news data
allNews=pd.read_pickle('/home/ec2-user/SageMaker/New Uncertainty/parsed_xml.pkl')
print(allNews.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 822737 entries, 0 to 822736
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   ID             822737 non-null  object        
 1   Title          822737 non-null  object        
 2   Type           822737 non-null  object        
 3   StartDate      822737 non-null  datetime64[ns]
 4   EndDate        822737 non-null  object        
 5   Text           822737 non-null  object        
 6   TextWordCount  822737 non-null  object        
 7   PubTitle       822737 non-null  object        
 8   SourceType     822737 non-null  object        
 9   Year           822737 non-null  int64         
 10  Month          822737 non-null  int64         
 11  Newspaper      822737 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(9)
memory usage: 75.3+ MB
None


In [4]:
# Merge with all news data
allNews['ID']=allNews['ID'].astype('int64')
df=df_regSentsExpandRelevant.merge(allNews[['ID','Title','Type','SourceType','StartDate','Newspaper','Year','Month']],
                                  on='ID',how='left')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 493418 entries, 0 to 493417
Data columns (total 22 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   ID                           493418 non-null  int64         
 1   RegSentsExpand               493418 non-null  object        
 2   NounChunksMatch              493418 non-null  int64         
 3   NounChunkMatchWords          493418 non-null  object        
 4   NounChunkMatchWordsFiltered  493418 non-null  object        
 5   NounChunkMatchFiltered       493418 non-null  int64         
 6   DominantNounChunk            493418 non-null  object        
 7   DominantNounChunkArea        493418 non-null  object        
 8   AllAreas                     493418 non-null  object        
 9   AreaCount                    493418 non-null  object        
 10  DominantArea                 493418 non-null  object        
 11  AllDistinctAreas          

In [5]:
print(df['Type'].value_counts())
print(df['SourceType'].value_counts())

News    493418
Name: Type, dtype: int64
Newspapers    493418
Name: SourceType, dtype: int64


In [6]:
# # Refine to sourcetype=newspaper
# df=df[df['SourceType']=='Newspapers'].reset_index(drop=True)
# print(df.info())

In [7]:
print(df.head())

           ID                                     RegSentsExpand  \
0   292620682  Now, one hopes, more attention will be paid to...   
1   307490698  His economic program as a candidate in the Rep...   
2   307420103  As the Bank of New England hearings suggest, t...   
3   307576786  The Justice Department contends that the Eight...   
4  1944990599  "The remittances from drivers alone reach almo...   

   NounChunksMatch                                NounChunkMatchWords  \
0                8  [land use, civil right, land use, land use, lo...   
1                2                                 [tax cut, tax cut]   
2               10  [new england, bank hold company, new england, ...   
3                4  [civil action, criminal activity, civil claim,...   
4                1                            [government regulation]   

                         NounChunkMatchWordsFiltered  NounChunkMatchFiltered  \
0  [land use, civil right, land use, land use, pu...                    

## 2. Create dummies for areas

In [8]:
# List of columns for different approaches
col_list=[]
for i in range(1,17):
    var1='DominantNounChunkArea'+str(i)
    col_list.append(var1)
    var2='DominantArea'+str(i)
    col_list.append(var2)
    var3='UniqueDistinctArea'+str(i)
    col_list.append(var3)
    var4='DominantDistinctArea'+str(i)
    col_list.append(var4)
col_list.append('DominantNounChunkArea8-9')
col_list.append('DominantArea8-9')
col_list.append('UniqueDistinctArea8-9')
col_list.append('DominantDistinctArea8-9')
#print(col_list)

### 2.1 Dummies by dominant noun chunk area (dnca)

In [9]:
for i in range(1,17):
    var='DominantNounChunkArea'+str(i)
    df[var]=0
    for j in range(0, len(df)):
        if i in df['DominantNounChunkArea'][j]:
            df[var][j]=1
print(df)

/home/ec2-user/SageMaker/.conda/envs/my_py/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


                ID                                     RegSentsExpand  \
0        292620682  Now, one hopes, more attention will be paid to...   
1        307490698  His economic program as a candidate in the Rep...   
2        307420103  As the Bank of New England hearings suggest, t...   
3        307576786  The Justice Department contends that the Eight...   
4       1944990599  "The remittances from drivers alone reach almo...   
...            ...                                                ...   
493413  2009071770  • A Feb. 28 KidsPost article about an upcoming...   
493414  1933811831  The Washington Post is committed to correcting...   
493415  2411004924  Kohl's has reopened most of its more than 1,10...   
493416  1950787340  Hadges left the DEA in May to join the pharmac...   
493417  2321287809  The building does not have carbon monoxide det...   

        NounChunksMatch                                NounChunkMatchWords  \
0                     8  [land use, civil rig

### 2.2 Dummies by dominant area (da)

In [10]:
for i in range(1,17):
    var='DominantArea'+str(i)
    df[var]=0
    for j in range(0, len(df)):
        if i in df['DominantArea'][j]:
            df[var][j]=1

/home/ec2-user/SageMaker/.conda/envs/my_py/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### 2.3 Dummies by distinct area (uda)

In [11]:
for i in range(1,17):
    var='UniqueDistinctArea'+str(i)
    df[var]=0
    for j in range(0, len(df)):
        if i in df['UniqueDistinctAreas'][j]:
            df[var][j]=1
#print(df.info())

/home/ec2-user/SageMaker/.conda/envs/my_py/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### 2.4 Dummies by dominant distinct area (dda)

In [12]:
for i in range(1,17):
    var='DominantDistinctArea'+str(i)
    df[var]=0
    for j in range(0, len(df)):
        if i in df['DominantDistinctArea'][j]:
            df[var][j]=1
#print(df.info())

/home/ec2-user/SageMaker/.conda/envs/my_py/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [13]:
# Combine areas 8 & 9
for var in ['DominantNounChunkArea','DominantArea','UniqueDistinctArea','DominantDistinctArea']:
    df[var+'8-9']=0
    df.loc[(df[var+'8']==1) | (df[var+'9']==1), var+'8-9']=1
print(df.head())

           ID                                     RegSentsExpand  \
0   292620682  Now, one hopes, more attention will be paid to...   
1   307490698  His economic program as a candidate in the Rep...   
2   307420103  As the Bank of New England hearings suggest, t...   
3   307576786  The Justice Department contends that the Eight...   
4  1944990599  "The remittances from drivers alone reach almo...   

   NounChunksMatch                                NounChunkMatchWords  \
0                8  [land use, civil right, land use, land use, lo...   
1                2                                 [tax cut, tax cut]   
2               10  [new england, bank hold company, new england, ...   
3                4  [civil action, criminal activity, civil claim,...   
4                1                            [government regulation]   

                         NounChunkMatchWordsFiltered  NounChunkMatchFiltered  \
0  [land use, civil right, land use, land use, pu...                    

## 3. Monthly article counts by area

In [14]:
# Aggregate monthly article counts
monthlyAreaCount=df[['Newspaper','Year','Month']+col_list].groupby(['Newspaper','Year','Month']).agg('sum').reset_index()
print(monthlyAreaCount.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2945 entries, 0 to 2944
Data columns (total 71 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Newspaper                 2945 non-null   object
 1   Year                      2945 non-null   int64 
 2   Month                     2945 non-null   int64 
 3   DominantNounChunkArea1    2945 non-null   int64 
 4   DominantArea1             2945 non-null   int64 
 5   UniqueDistinctArea1       2945 non-null   int64 
 6   DominantDistinctArea1     2945 non-null   int64 
 7   DominantNounChunkArea2    2945 non-null   int64 
 8   DominantArea2             2945 non-null   int64 
 9   UniqueDistinctArea2       2945 non-null   int64 
 10  DominantDistinctArea2     2945 non-null   int64 
 11  DominantNounChunkArea3    2945 non-null   int64 
 12  DominantArea3             2945 non-null   int64 
 13  UniqueDistinctArea3       2945 non-null   int64 
 14  DominantDistinctArea3   

In [15]:
print(monthlyAreaCount.head())

      Newspaper  Year  Month  DominantNounChunkArea1  DominantArea1  \
0  Boston Globe  1985      1                      21             13   
1  Boston Globe  1985      2                      29             24   
2  Boston Globe  1985      3                      34             21   
3  Boston Globe  1985      4                      32             23   
4  Boston Globe  1985      5                      37             25   

   UniqueDistinctArea1  DominantDistinctArea1  DominantNounChunkArea2  \
0                   13                      7                      21   
1                   12                     12                      27   
2                   15                     12                      29   
3                   12                      9                      33   
4                   15                     14                      32   

   DominantArea2  UniqueDistinctArea2  ...  UniqueDistinctArea15  \
0             15                   13  ...                     4  

In [16]:
monthlyAreaCount.to_csv('/home/ec2-user/SageMaker/New Uncertainty/Categorical Index/RegArea_MonthlyArticleCountByNewspaper.csv',index=False)

## 4. Merge with sentiment scores

In [17]:
# Merge with sentiment scores
sentimentScores=pd.read_csv('/home/ec2-user/SageMaker/New Uncertainty/Sentiment Analysis/RegRelevant_ArticleSentimentScores.csv')
print(sentimentScores.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493418 entries, 0 to 493417
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ID                493418 non-null  int64  
 1   StartDate         493418 non-null  object 
 2   Newspaper         493418 non-null  object 
 3   UncertaintyScore  493418 non-null  float64
 4   GIscore           493418 non-null  float64
 5   LMscore           493418 non-null  float64
 6   LSDscore          493418 non-null  float64
dtypes: float64(4), int64(1), object(2)
memory usage: 26.4+ MB
None


In [18]:
# Merge
df['ID']=df['ID'].astype('int64')
df2=df.merge(sentimentScores[['ID','UncertaintyScore','GIscore','LMscore','LSDscore']],on='ID',how='left')
#print(df2.info())

In [19]:
df2[['ID','StartDate','Newspaper','UncertaintyScore','GIscore','LMscore','LSDscore']+col_list].to_csv('/home/ec2-user/SageMaker/New Uncertainty/Categorical Index/RegArea_ArticleSentimentScores.csv',index=False)

In [20]:
# Calculate average monthly scores by area
df_reg=df2
df_reg['ArticleCount']=1
monthlyAreaScores=df_reg[['Year','Month','ArticleCount']].\
    groupby(['Year','Month']).agg({'ArticleCount':'sum'}).reset_index()
print(monthlyAreaScores.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428 entries, 0 to 427
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   Year          428 non-null    int64
 1   Month         428 non-null    int64
 2   ArticleCount  428 non-null    int64
dtypes: int64(3)
memory usage: 10.2 KB
None


In [21]:
for var in col_list:
    df_area=df_reg[df_reg[var]==1].reset_index(drop=True)
    areaScore=df_area[['Year','Month','UncertaintyScore','GIscore','LSDscore','LMscore','ArticleCount']].\
                                    groupby(['Year','Month']).agg({'UncertaintyScore':'mean','GIscore':'mean',
                                   'LSDscore':'mean','LMscore':'mean','ArticleCount':'sum'}).reset_index()
    areaScore=areaScore.rename(columns={'UncertaintyScore':'UncertaintyScore'+'_'+var,'GIscore':'GIscore'+'_'+var,
                                       'LSDscore':'LSDscore'+'_'+var,'LMscore':'LMscore'+'_'+var,'ArticleCount':'ArticleCount'+'_'+var})
    monthlyAreaScores=monthlyAreaScores.merge(areaScore,on=['Year','Month'],how='left')
print(monthlyAreaScores.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 428 entries, 0 to 427
Columns: 343 entries, Year to ArticleCount_DominantDistinctArea8-9
dtypes: float64(276), int64(67)
memory usage: 1.1 MB
None


In [22]:
monthlyAreaScores.to_csv('/home/ec2-user/SageMaker/New Uncertainty/Categorical Index/RegArea_MonthlySentimentScores.csv', index=False)

## A1. Examine the results

In [ ]:
# Investigate the results
print(df2.info())

In [ ]:
# Most negative/positive articles in specified area
temp=df2[(df2['DominantDistinctArea3']==1) & (df2['Year']==2014) & (df2['Month']==10)]
temp=temp.sort_values('LMscore').reset_index(drop=True)
print(temp[['ID','LMscore']])

In [ ]:
for i in range(0,len(temp)):
    print(temp['ID'][i],temp['Title'][i],':',temp['Newspaper'][i],'\n',temp['RegSentsExpand'][i],temp['NounChunkMatchWordsFiltered'][i],
          temp['AllDistinctAreas'][i], temp['DominantDistinctArea'][i],temp['LMscore'][i],'\n')

In [ ]:
for i in range(0,len(df)):
    if  (df['Year'][i]==2002) & (df['Month'][i]==7):
        print(df['ID'][i],df['Title'][i])